In [7]:
%load_ext autoreload
%autoreload 2

import json
import rtcloud

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
rtcloud.init('0.0.0.0:12345', conf={})
rtcloud.upload()
rtcloud.watch('1234')

NameError: name 'SERVER_ADDRESS' is not defined